<a href="https://colab.research.google.com/github/Ebasurtos/AHPC/blob/main/M%C3%A9tricas_y_Medidas_de_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# performance_metrics.py
import numpy as np

class PerformanceMetrics:
    """Clase para calcular métricas de performance paralela"""

    @staticmethod
    def calculate_strong_scaling_efficiency(T1, Tp, p):
        """Eficiencia de escalabilidad fuerte"""
        return T1 / (p * Tp)

    @staticmethod
    def calculate_weak_scaling_efficiency(T1, Tp, p, scaled_N):
        """Eficiencia de escalabilidad débil"""
        # T1: tiempo con problema base, 1 proceso
        # Tp: tiempo con problema escalado, p procesos
        return T1 / Tp

    @staticmethod
    def calculate_karp_flatt_metric(p, speedup):
        """Métrica de Karp-Flat para medir overhead serial"""
        if p == 1:
            return 0
        return (1/speedup - 1/p) / (1 - 1/p)

    @staticmethod
    def calculate_iso_efficiency(p, efficiency, alpha=0.01):
        """Función de iso-eficiencia"""
        # alpha: fracción de overhead de comunicación
        return p * efficiency / (1 - efficiency - alpha * np.sqrt(p))

    @staticmethod
    def analyze_bottlenecks(comp_time, comm_time, sync_time, total_time):
        """Análisis de cuellos de botella"""
        metrics = {
            'comp_percentage': comp_time / total_time * 100,
            'comm_percentage': comm_time / total_time * 100,
            'sync_percentage': sync_time / total_time * 100,
            'idle_percentage': 100 - (comp_time + comm_time + sync_time) / total_time * 100
        }

        # Diagnóstico
        if metrics['comm_percentage'] > 30:
            metrics['bottleneck'] = 'Communication'
        elif metrics['sync_percentage'] > 20:
            metrics['bottleneck'] = 'Synchronization'
        elif metrics['idle_percentage'] > 10:
            metrics['bottleneck'] = 'Load Imbalance'
        else:
            metrics['bottleneck'] = 'Computation'

        return metrics

    @staticmethod
    def calculate_scalability_class(speedups, p_values):
        """Clasifica el tipo de escalabilidad"""
        ideal = p_values
        actual = np.array(speedups)

        # Calcular coeficiente de determinación R²
        r_squared = 1 - np.sum((actual - ideal)**2) / np.sum((actual - np.mean(ideal))**2)

        if r_squared > 0.95:
            return "Super-linear", r_squared
        elif r_squared > 0.85:
            return "Linear", r_squared
        elif r_squared > 0.70:
            return "Sub-linear", r_squared
        else:
            return "Poor", r_squared

# Ejemplo de uso
if __name__ == "__main__":
    # Datos de ejemplo
    T1 = 100.0  # tiempo con 1 proceso
    T4 = 30.0   # tiempo con 4 procesos
    T16 = 12.0  # tiempo con 16 procesos

    p_values = [1, 4, 16]
    speedups = [1, T1/T4, T1/T16]

    metrics = PerformanceMetrics()

    print("Análisis de métricas de performance:")
    print(f"Speedup con 4 procesos: {speedups[1]:.2f}")
    print(f"Eficiencia fuerte: {metrics.calculate_strong_scaling_efficiency(T1, T4, 4):.2%}")
    print(f"Métrica Karp-Flatt: {metrics.calculate_karp_flatt_metric(4, speedups[1]):.4f}")

    bottleneck = metrics.analyze_bottlenecks(
        comp_time=20, comm_time=8, sync_time=2, total_time=30
    )
    print(f"Cuello de botella identificado: {bottleneck['bottleneck']}")